<h1><center> Data Cleaning, Preporcessing & Engineering </center></h1>

#### Libraries & Data

In [1]:
import pandas as pd
import numpy as np

np.random.seed(42)

In [2]:
riaa_df_good = pd.read_csv('../00_Data/riaa_songs_df_with_ids.csv')
df = pd.read_csv('../00_Data/df_merged.csv')
gf_df = pd.read_csv('../00_Data/genre_followers.csv' )
ex_df = pd.read_csv('../00_Data/explicit.csv' )

### Drop Duplicates & Merge my RIAA songs with complete data frame

- Reason I am dropping duplicates using a subset is becasue when obtaining my RIAA Songs some songs were duplicated but somehow had different values in one column , so they were not being dropped. Dropping using a subset of track_id will return all unique Track_id's which will get rid of those duplicate rows.

In [3]:
riaa_df_good.drop_duplicates(subset = 'track_id' , inplace = True)

In [4]:
df_good = pd.merge(df , riaa_df_good[['award' , 'track_id']] , how = 'left' )

### Award Binary

- Since I only care about if a track went at least Gold( Which is represented in Award column as '0'). I will loop through my award column and for any number zero or greater it will append a '1' for my positive class & '0' for my negative class(which is represented in Award column as 'NaN'.

In [5]:
y_new = []
for i in df_good.award:
    if i  >= 0:
        y_new.append(1)
    else:
        y_new.append(0)
df_good['award_binary'] = pd.DataFrame(y_new)

### Feature Engineering 

- The duration feature was orginally acquired in milliseconds. Since interpreting time in seconds is easier, I decided to divide each number by 1000 to convert the time into seconds.

In [6]:
df_good['duration'] = df_good.duration_ms / 1000
df_good.drop(columns = 'duration_ms' , inplace = True)

### Concatenating 

In [7]:
df_good = pd.concat([df_good , ex_df] , axis = 1 )
df_good.drop(columns = 'id' , inplace  = True)

In [8]:
df_good.drop_duplicates(subset = 'track_id' , inplace = True)

### Merge

In [9]:
df_good = pd.merge(df_good , gf_df , how = 'left' , left_on = 'artist' , right_on= 'artists')

In [10]:
artist_genres = df_good[['artist', 'genres']].drop_duplicates()

### Preprocessing Genres

In [11]:
artist_genres.head(2)

,artist,genres
0,The Chainsmokers,"['edm', 'electropop', 'pop', 'tropical house']"
19,Anuel Aa,"['reggaeton flow', 'trap latino']"


- As seen above you can see that my Genres for each artist are strings in a list, I need to take out each genre & and put it in its own column. How I did this was by creating a dictionary for each genre and zipping each genre to its respective column. So for each Artist there was 627 Genre columns, If the artist perform a genre then a '1' would appear under that column, & if not then 'NaN' would appear

In [12]:
artist_genres = artist_genres[artist_genres.genres != '[]']

In [13]:
artist_genres.dropna(inplace=True)

In [14]:
artist_genres['genres'] = artist_genres['genres'].map(lambda x: eval(x))

In [15]:
artist_genre_dummies = pd.DataFrame(dict(zip([*artist_genres.artist], 
[*artist_genres.genres.map(lambda x: dict(zip(x, [1]*len(x))))]))).T

### Merge all data frames together

In [16]:
df_complete_all = pd.merge(df_good , artist_genre_dummies , how = 'left' , left_on = 'artist' , right_on= artist_genre_dummies.index)

- Duplicate columns

In [17]:
df_complete_all.drop(columns = ['artists','genres'] , inplace = True)

In [18]:
df_complete_all.fillna(value = 0 , inplace = True)

### 2000-2018

- Since I only collected RIAA songs from 2000-2018 I want my other songs to represent that time period also. So I am setting my 'release_date' column as a datetime & putting it as my index so I can filter out songs before 2000.

In [19]:
df_complete_all['release_date'] = pd.to_datetime(df_complete_all.release_date)

In [20]:
df_complete_present= df_complete_all.set_index(df_complete_all.release_date)

In [21]:
df_2000_2018 = df_complete_present['2000':'2018']

In [22]:
df_2000_2018.drop(columns = 'release_date' , inplace = True)

/Users/benjaminnew/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


### Export to CSV

In [23]:
df_2000_2018.to_csv('../00_Data/df_complete.csv')